## Imports and Setup

In [1]:
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import os 
os.chdir('/home/itingzon.unicef/giga')
from src import sat_download
from utils import data_utils
from utils import config_utils
from utils import pred_utils

%load_ext autoreload
%autoreload 2

## Load Config Files

In [4]:
iso_code="MOZ"
adm_level="ADM2"
sum_threshold=5
buffer_size=150
spacing=150

config_file = os.path.join(os.getcwd(), "configs/data_configs/data_config_ISO_AF.yaml")
sat_config_file = os.path.join(os.getcwd(), "configs/sat_configs/sat_config_500x500_60cm.yaml")
sat_creds_file = os.path.join(os.getcwd(), "configs/sat_configs/issa_sat_creds.yaml")

config = config_utils.load_config(config_file)
sat_config = config_utils.load_config(sat_config_file)
sat_creds = config_utils.create_config(sat_creds_file)

## Generate Prediction Tiles

In [3]:
tiles = pred_utils.batch_process_tiles(
    config, iso_code, spacing, buffer_size, sum_threshold, adm_level
)

INFO:root:Gile 0/159
INFO:root:Total tiles: (38016, 5)
INFO:root:Gurue 1/159
INFO:root:Total tiles: (79514, 5)
INFO:root:Ile 2/159
INFO:root:Total tiles: (69742, 5)
INFO:root:Inhassunge 3/159
INFO:root:Total tiles: (8116, 5)
INFO:root:Luabo 4/159
INFO:root:Total tiles: (6264, 5)
INFO:root:Lugela 5/159
INFO:root:Total tiles: (53263, 5)
INFO:root:Maganja Da Costa 6/159
INFO:root:Total tiles: (23519, 5)
INFO:root:Maquival 7/159
INFO:root:Total tiles: (9541, 5)
INFO:root:Milange 8/159
INFO:root:Total tiles: (93184, 5)
INFO:root:Mocuba 9/159
INFO:root:Total tiles: (75781, 5)
INFO:root:Mocubela 10/159
INFO:root:Total tiles: (17821, 5)
INFO:root:Molumbo 11/159
INFO:root:Total tiles: (66708, 5)
INFO:root:Mopeia 12/159
INFO:root:Total tiles: (22960, 5)
INFO:root:Morrumbala 13/159
INFO:root:Total tiles: (61467, 5)
INFO:root:Mulevala 14/159
INFO:root:Total tiles: (35363, 5)
INFO:root:Namacurra 15/159
INFO:root:Total tiles: (24330, 5)
INFO:root:Namarroi 16/159
INFO:root:Total tiles: (48293, 5)
INF

## Download Satellite Images

In [6]:
pred_utils.batch_download_sat_images(
    sat_config, sat_creds, config, iso_code, spacing, buffer_size, sum_threshold, adm_level
)

INFO:root:Data dimensions: (9732, 7), CRS: EPSG:3857


Ancuabe 1/159 total tiles: (9732, 5)


INFO:root:Data dimensions: (8563, 7), CRS: EPSG:3857


Balama 2/159 total tiles: (8563, 5)


INFO:root:Data dimensions: (13158, 7), CRS: EPSG:3857


...

Namacurra 156/159 total tiles: (24330, 5)


INFO:root:Data dimensions: (48293, 7), CRS: EPSG:3857


Namarroi 157/159 total tiles: (48293, 5)
Nicoadala 158/159 total tiles: (19635, 5)


INFO:root:Data dimensions: (19635, 7), CRS: EPSG:3857
INFO:root:Data dimensions: (33891, 7), CRS: EPSG:3857


Pebane 159/159 total tiles: (33891, 5)


## Run Model

In [ ]:
model_config_file = os.path.join(cwd, f"configs/cnn_configs/vgg16.yaml")
model_config = config_utils.load_config(model_config_file)
result = pred_utils.cnn_predict(tiles, iso_code, shapename, model_config, sat_dir, n_classes=2)